# Linking drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# imports

In [3]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os

# Loading the dataset

In [4]:
ratings = pd.read_csv('/content/drive/MyDrive/recommendation system/movie recommendation/archive/ratings.csv')
movies = pd.read_csv('/content/drive/MyDrive/recommendation system/movie recommendation/archive/movies_metadata.csv')
keywords = pd.read_csv('/content/drive/MyDrive/recommendation system/movie recommendation/archive/keywords.csv')
credits = pd.read_csv('/content/drive/MyDrive/recommendation system/movie recommendation/archive/credits.csv')

<ipython-input-4-84499477014f>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('/content/drive/MyDrive/recommendation system/movie recommendation/archive/movies_metadata.csv')


# EDA

##ratings

In [5]:
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
5,1,1968,4.0,1425942148
6,1,2762,4.5,1425941300
7,1,2918,5.0,1425941593
8,1,2959,4.0,1425941601
9,1,4226,4.0,1425942228


In [6]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


### Checking for null values in ratings

In [7]:
for i in ratings.columns:
  print(i, ratings[i].isnull().any())

userId False
movieId False
rating False
timestamp False


### checking for duplicate values

In [8]:
ratings[ratings.duplicated(subset = ['userId', "movieId"])].count()

,0
userId,0
movieId,0
rating,0
timestamp,0


## Removing users who have rated less than 50 movies

In [14]:
df = ratings.groupby("userId")["movieId"].count()

df = df[df >= 50]
ratings = ratings[ratings["userId"].isin(df.index)]
ratings.reset_index(drop = True, inplace = True)

In [16]:
ratings.head()

,userId,movieId,rating,timestamp
0,4,223,4.0,1042668576
1,4,415,4.0,1042667925
2,4,648,4.0,1042674800
3,4,1097,5.0,1042667925
4,4,1197,4.0,1042667956


## Droping movies which have been rated by less than 100 users

In [18]:
df = ratings.groupby("movieId")["userId"].count()
df = df[df >= 200]
ratings = ratings[ratings["movieId"].isin(df.index)]
ratings.reset_index(drop = True, inplace = True)

In [19]:
ratings

,userId,movieId,rating,timestamp
0,4,223,4.0,1042668576
1,4,415,4.0,1042667925
2,4,648,4.0,1042674800
3,4,1097,5.0,1042667925
4,4,1197,4.0,1042667956
...,...,...,...,...
22119527,270896,56367,4.5,1257031529
22119528,270896,58559,5.0,1257031564
22119529,270896,60069,5.0,1257032032
22119530,270896,63082,4.5,1257031764


## movies

In [ ]:
movies.head(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0


In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

### Drop irrelevant columns

In [ ]:
movies.drop(columns = ["adult", "belongs_to_collection", "budget", "homepage", "imdb_id", "original_language", "original_title", "production_companies", "production_countries", "release_date",  "runtime", "spoken_languages", "status", "video"], inplace = True)

In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        45466 non-null  object 
 1   id            45466 non-null  object 
 2   overview      44512 non-null  object 
 3   popularity    45461 non-null  object 
 4   poster_path   45080 non-null  object 
 5   revenue       45460 non-null  float64
 6   tagline       20412 non-null  object 
 7   title         45460 non-null  object 
 8   vote_average  45460 non-null  float64
 9   vote_count    45460 non-null  float64
dtypes: float64(3), object(7)
memory usage: 3.5+ MB


### Reformating the columns to be of correct types

In [ ]:
movies['id'] = pd.to_numeric(movies['id'], errors='coerce').astype('Int64')
movies['popularity'] = pd.to_numeric(movies['popularity'], errors='coerce').astype('float64')
movies["poster_path"] = movies["poster_path"].astype("string")
movies["tagline"] = movies["tagline"].astype("string")
movies["title"] = movies["title"].astype("string")
movies["vote_average"] = movies["vote_average"].astype("float64")
movies["vote_count"] = movies["vote_count"].astype("float64")

In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        45466 non-null  object 
 1   id            45463 non-null  Int64  
 2   overview      44512 non-null  object 
 3   popularity    45460 non-null  float64
 4   poster_path   45080 non-null  string 
 5   revenue       45460 non-null  float64
 6   tagline       20412 non-null  string 
 7   title         45460 non-null  string 
 8   vote_average  45460 non-null  float64
 9   vote_count    45460 non-null  float64
dtypes: Int64(1), float64(4), object(2), string(3)
memory usage: 3.5+ MB


### Combining overview and tagline as description

In [ ]:
movies["overview"] = movies["overview"].fillna("")
movies["tagline"] = movies["tagline"].fillna("")

In [ ]:
movies["description"] = movies["overview"] + " " + movies["tagline"]

### splitting genres to make a list

In [ ]:
def split_genre(input):
  input = eval(input)
  genre = []
  for i in input:
    genre.append(i['name'])
  return genre

movies['genres'] = movies['genres'].apply(split_genre)
movies.head()

,genres,id,overview,popularity,poster_path,revenue,tagline,title,vote_average,vote_count,description
0,"[Animation, Comedy, Family]",862,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,373554033.0,<NA>,Toy Story,7.7,5415.0,<NA>
1,"[Adventure, Fantasy, Family]",8844,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,262797249.0,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,When siblings Judy and Peter discover an encha...
2,"[Romance, Comedy]",15602,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,0.0,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,A family wedding reignites the ancient feud be...
3,"[Comedy, Drama, Romance]",31357,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,81452156.0,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"Cheated on, mistreated and stepped on, the wom..."
4,[Comedy],11862,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,76578911.0,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Just when George Banks has recovered from his ...


### Renaming id to movieId

In [ ]:
movies.rename(columns = {"id": "movieId"}, inplace = True)

### Removing null values

In [ ]:
movies.dropna(subset = ["movieId", "title", "poster_path"], inplace = True)

In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45077 entries, 0 to 45465
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        45077 non-null  object 
 1   movieId       45077 non-null  Int64  
 2   overview      44163 non-null  object 
 3   popularity    45077 non-null  float64
 4   poster_path   45077 non-null  string 
 5   revenue       45077 non-null  float64
 6   tagline       20397 non-null  string 
 7   title         45077 non-null  string 
 8   vote_average  45077 non-null  float64
 9   vote_count    45077 non-null  float64
 10  description   20389 non-null  string 
dtypes: Int64(1), float64(4), object(2), string(4)
memory usage: 4.2+ MB


### Removing duplicate values

In [ ]:
movies.drop_duplicates(subset=["title"],inplace = True)

In [ ]:
print(movies['title'].nunique())
print(movies['movieId'].nunique())

41914
41914


In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41914 entries, 0 to 45465
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        41914 non-null  object 
 1   movieId       41914 non-null  Int64  
 2   overview      41039 non-null  object 
 3   popularity    41914 non-null  float64
 4   poster_path   41914 non-null  string 
 5   revenue       41914 non-null  float64
 6   tagline       19043 non-null  string 
 7   title         41914 non-null  string 
 8   vote_average  41914 non-null  float64
 9   vote_count    41914 non-null  float64
 10  description   19035 non-null  string 
dtypes: Int64(1), float64(4), object(2), string(4)
memory usage: 3.9+ MB


## Keywords

In [ ]:
keywords.head(10)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
5,949,"[{'id': 642, 'name': 'robbery'}, {'id': 703, '..."
6,11860,"[{'id': 90, 'name': 'paris'}, {'id': 380, 'nam..."
7,45325,[]
8,9091,"[{'id': 949, 'name': 'terrorist'}, {'id': 1562..."
9,710,"[{'id': 701, 'name': 'cuba'}, {'id': 769, 'nam..."


In [ ]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


### Rename id to movieID and convert to integer

In [ ]:
keywords.rename(columns = {"id": "movieId"}, inplace = True)

In [ ]:
keywords["movieId"] = keywords["movieId"].astype("Int64")

### Dropping null and duplicated values

In [ ]:
keywords.dropna(subset = ["movieId"], inplace = True)

In [ ]:
keywords.drop_duplicates(subset = ["movieId"], inplace = True)

In [ ]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45432 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movieId   45432 non-null  Int64 
 1   keywords  45432 non-null  object
dtypes: Int64(1), object(1)
memory usage: 1.1+ MB


### Converting keywords into a list

In [ ]:
def convert_keywords(input):
  input = eval(input)
  keywords = []
  for i in input:
    keywords.append(i['name'])
  return keywords

keywords["keywords"]= keywords['keywords'].apply(convert_keywords)
keywords.head()

,movieId,keywords
0,862,"[jealousy, toy, boy, friendship, friends, riva..."
1,8844,"[board game, disappearance, based on children'..."
2,15602,"[fishing, best friend, duringcreditsstinger, o..."
3,31357,"[based on novel, interracial relationship, sin..."
4,11862,"[baby, midlife crisis, confidence, aging, daug..."


## credits

In [ ]:
credits.head(10)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
5,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",949
6,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...",11860
7,"[{'cast_id': 2, 'character': 'Tom Sawyer', 'cr...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",45325
8,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",9091
9,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...",710


In [ ]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


### Rename id to movieId and convert it to int64

In [ ]:
credits.rename(columns = {"id" : "movieId"}, inplace = True)

In [ ]:
credits["movieId"] = credits["movieId"].astype("int64")

### extracting top 5 actors in the cast

In [ ]:
def get_cast(input):
  input = eval(input)
  cast = []
  for i in input:
    cast.append(i['name'])
  return cast[:5]

credits["cast"] =  credits['cast'].apply(get_cast)
credits.head()

,cast,crew,movieId
0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[Whitney Houston, Angela Bassett, Loretta Devi...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


### Extracting director from the crew

In [ ]:
def get_director(input):
  input = eval(input)
  for i in input:
    if(i['job'] == 'Director'):
      return i['name']
  return np.nan

credits["director"] =  credits['crew'].apply(get_director)
credits.head()

,cast,crew,movieId,director
0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,John Lasseter
1,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,Joe Johnston
2,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,Howard Deutch
3,"[Whitney Houston, Angela Bassett, Loretta Devi...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,Forest Whitaker
4,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,Charles Shyer


In [ ]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cast      45476 non-null  object
 1   crew      45476 non-null  object
 2   movieId   45476 non-null  int64 
 3   director  44589 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


### Dropping null and duplicated values

In [ ]:
credits.drop_duplicates(subset = ["movieId"], inplace = True)

In [ ]:
credits.dropna(subset = ["movieId"], inplace = True)

In [ ]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45432 entries, 0 to 45475
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cast      45432 non-null  object
 1   crew      45432 non-null  object
 2   movieId   45432 non-null  int64 
 3   director  44545 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.7+ MB


## general

In [ ]:
n_users = ratings["userId"].nunique()
n_movies = ratings["movieId"].nunique()
print(f"Number of unique users: {n_users}")
print(f"Number of unique movies: {n_movies}")

Number of unique users: 270896
Number of unique movies: 45115


In [ ]:
users_mean_ratings = ratings.groupby("userId")["rating"].mean()
movies_mean_ratings = ratings.groupby("movieId")["rating"].mean()
print(f"Mean rating given by users: {users_mean_ratings.mean()}")
print(f"Mean rating of movies: {movies_mean_ratings.mean()}")

Mean rating given by users: 3.620538206229964
Mean rating of movies: 3.061294172517874


In [ ]:
# fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# sns.boxplot(data=users_mean_ratings, ax=axes[0])
# axes[0].set_title("Boxplot of Mean Ratings given by each users")

# sns.countplot(x=users_mean_ratings, ax=axes[1])
# axes[1].set_title("Countplot of Mean Ratings given to each movie")
# axes[1].set_ylim(0, 5)

# # sns.boxplot(data = ratings["rating"], ax = axes[1])
# # axes[1].set_title("Boxplot of ratings")

# # plt.tight_layout()
# plt.show()

In [ ]:
# fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# sns.boxplot(data=movies_mean_ratings, ax=axes[0])
# axes[0].set_title("Boxplot of Mean Ratings")

# # sns.countplot(data=movies_mean_ratings, ax=axes[1])
# # axes[1].set_title("Countplot of Mean Ratings")

# plt.tight_layout()
# plt.show()

### Calculating baysian averaage


we calculate the average rating using **Bayesian Average**, given by:

$$
\text{avg_rating} = w_i \cdot A_i + (1 - w_i) \cdot S
$$

Where:
$$
 w_i = \frac{n_i}{n_i + n_{\text{avg}}}
$$
- \( n_i \): Total number of ratings given to movie \( i \)
- \( n_avg \): Average number of ratings given to a movie
- \( A_i \): Average rating of movie \( i \)
- \( S \): Mean average rating


In [ ]:
n_i = ratings.groupby("movieId")["rating"].count()
print("\nn_i: \n" , n_i)
n_avg = ratings.groupby("movieId")["rating"].count().mean()
print("\nnavg: \n",n_avg)
w = n_i / (n_i + n_avg)
print("\nw: \n", w)
A = ratings.groupby("movieId")["rating"].mean()
print("\n A: \n",A)
S = ratings["rating"].mean()
print("\nS: \n", S)
baysian_avg = w * A + (1 - w) * S
print("\n baysian_avg: \n", baysian_avg)


n_i: 
 movieId
1         66008
2         26060
3         15497
4          2981
5         15258
          ...  
176267        1
176269        1
176271        1
176273        1
176275        1
Name: rating, Length: 45115, dtype: int64

navg: 
 576.8433780339134

w: 
 movieId
1         0.991337
2         0.978344
3         0.964113
4         0.837867
5         0.963571
            ...   
176267    0.001731
176269    0.001731
176271    0.001731
176273    0.001731
176275    0.001731
Name: rating, Length: 45115, dtype: float64

 A: 
 movieId
1         3.888157
2         3.236953
3         3.175550
4         2.875713
5         3.079565
            ...   
176267    4.000000
176269    3.500000
176271    5.000000
176273    1.000000
176275    3.000000
Name: rating, Length: 45115, dtype: float64

S: 
 3.5280903543608817

 baysian_avg: 
 movieId
1         3.885038
2         3.243258
3         3.188202
4         2.981485
5         3.095904
            ...   
176267    3.528907
176269    3.528042
17

In [ ]:
baysian_avg_df = baysian_avg.reset_index()
baysian_avg_df.columns = ['movieId', 'baysian_avg']
movies = movies.merge(baysian_avg_df, on='movieId', how='left')

In [ ]:
movies.head()

,genres,movieId,overview,popularity,poster_path,revenue,tagline,title,vote_average,vote_count,description,baysian_avg
0,"[Animation, Comedy, Family]",862,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,373554033.0,<NA>,Toy Story,7.7,5415.0,<NA>,3.555954
1,"[Adventure, Fantasy, Family]",8844,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,262797249.0,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,When siblings Judy and Peter discover an encha...,3.568878
2,"[Romance, Comedy]",15602,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,0.0,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,A family wedding reignites the ancient feud be...,NaN
3,"[Comedy, Drama, Romance]",31357,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,81452156.0,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"Cheated on, mistreated and stepped on, the wom...",NaN
4,[Comedy],11862,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,76578911.0,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Just when George Banks has recovered from his ...,NaN


In [ ]:
movies.sort_values("baysian_avg", ascending=False).head(10)

,genres,movieId,overview,popularity,poster_path,revenue,tagline,title,vote_average,vote_count,description,baysian_avg
3959,"[Drama, Thriller]",318,The Million Dollar Hotel starts with a jump fr...,4.938231,/4wxOzpCjXY2RQgS3YoHuvKdItTF.jpg,0.0,<NA>,The Million Dollar Hotel,5.9,76.0,<NA>,4.423345
534,"[Comedy, Drama, Romance]",858,A young boy who tries to set his dad up on a d...,10.234919,/afkYP15OeUOD0tFEmj6VvejuOcz.jpg,227799884.0,"What if someone you never met, someone you nev...",Sleepless in Seattle,6.5,630.0,A young boy who tries to set his dad up on a d...,4.331688
286,[Drama],527,A drama about a Maori family lving in Auckland...,4.025276,/8QF48QS5lvtrhOclt22RFz3bSQm.jpg,2201126.0,"A family in crisis, a life in chaos... Nothing...",Once Were Warriors,7.6,106.0,A drama about a Maori family lving in Auckland...,4.260288
459,"[Action, Adventure, Crime, Thriller]",2019,"When a woman's father goes missing, she enlist...",7.834351,/6WEu60V7EzncuFJSVmGJzhFvs4I.jpg,74189677.0,Don't hunt what you can't kill.,Hard Target,6.1,237.0,"When a woman's father goes missing, she enlist...",4.226293
11513,[Comedy],2959,"Newly engaged, Ben and Sadie can't wait to sta...",7.102076,/9I8NiY5cfFmwgI8Y5NKQqsuDawS.jpg,69307224.0,First came love... then came Reverend Frank.,License to Wed,5.3,258.0,"Newly engaged, Ben and Sadie can't wait to sta...",4.224028
8319,"[Drama, Crime, Mystery, Comedy]",750,Miss Marple believes she’s seen a murder in a ...,7.261845,/iniMlncVkZzQTZ3kLU6eEtoZz0V.jpg,0.0,Who is the blonde in the box?,Murder She Said,7.0,31.0,Miss Marple believes she’s seen a murder in a ...,4.199339
8500,"[Romance, Comedy]",5618,Two distant cousins meet at a wedding banquet ...,0.624726,/z3iGJixiABlkXbiloRQWHsrIIFB.jpg,0.0,<NA>,"Cousin, Cousine",7.3,4.0,<NA>,4.184435
34965,[Drama],58559,"Paris, 1830: Octave, betrayed by his mistress,...",0.778480,/a1CXNSJpvF4WLKAfjzyyJ6GUB3m.jpg,0.0,<NA>,Confession of a Child of the Century,4.9,6.0,<NA>,4.172681
6031,"[Crime, Drama, Thriller]",6016,A compulsive gambler plans the heist of his li...,4.022761,/qGadykMllvTf3EE1LMQ51g4wGQ0.jpg,0.0,He doesn't want money. He wants what money can...,The Good Thief,6.0,32.0,A compulsive gambler plans the heist of his li...,4.169329
3010,"[Thriller, Crime, Drama]",1213,Tom Ripley is a calculating young man who beli...,11.655020,/cTV9p4Pup3XLGYZ0aNJCF9c4GuL.jpg,128798265.0,How far would you go to become someone else?,The Talented Mr. Ripley,7.1,790.0,Tom Ripley is a calculating young man who beli...,4.167437


In [ ]:
movies.sort_values("baysian_avg", ascending=True).head(10)

,genres,movieId,overview,popularity,poster_path,revenue,tagline,title,vote_average,vote_count,description,baysian_avg
10949,"[Adventure, Horror, Science Fiction]",3593,Four explorers are summoned to Peru by the bri...,0.662919,/5bUdwNdumc2cPzq9dQIkYMxovQx.jpg,0.0,<NA>,Dr. Cyclops,6.1,15.0,<NA>,1.813569
18408,[Drama],1556,A feverish collision of avant-garde aesthetics...,3.502077,/v2HfNo9eLNHvN6Rh9BQiJBVGDPQ.jpg,0.0,The trials and tribulations of Eddie and other...,Funeral Parade of Roses,7.1,21.0,A feverish collision of avant-garde aesthetics...,2.088090
10894,"[Drama, Adventure, Science Fiction, Romance]",1381,"Spanning over one thousand years, and three pa...",11.577121,/lGIJvQ8ZQazOHVAA0E34yIy7gxv.jpg,15304890.0,Death is the road to awe,The Fountain,6.8,852.0,"Spanning over one thousand years, and three pa...",2.177884
3241,"[Adventure, Drama, Family]",2383,An orphan bear cub hooks up with an adult male...,4.158572,/1EgJJ7uIsMYZJ69T6KfQunuLLBk.jpg,138241022.0,He's an orphan... at the start of a journey. A...,The Bear,6.9,85.0,An orphan bear cub hooks up with an adult male...,2.190507
3268,"[Action, Adventure, Comedy, Family, Fantasy, S...",1499,The four turtles travel back in time to the da...,8.136880,/bax6hItUe0XSeyO25J5V7hhgkkZ.jpg,42000000.0,The Turtles Are Back....In Time.,Teenage Mutant Ninja Turtles III,5.1,189.0,The four turtles travel back in time to the da...,2.194967
10310,[Drama],546,Bree is about to get a sex change operation th...,7.398205,/ehYc5DRe1Ipnn5XSf3Tol3LMbrq.jpg,13350369.0,Life is a Journey. Bring an Open Mind.,Transamerica,6.9,149.0,Bree is about to get a sex change operation th...,2.209079
11419,"[Fantasy, Adventure, Animation, Comedy, Family]",810,The King of Far Far Away has died and Shrek an...,14.072479,/lA17Mlz8hXVZsW5Y3WuUa8KPilQ.jpg,798958165.0,Who's ready for Thirds?,Shrek the Third,6.0,2355.0,The King of Far Far Away has died and Shrek an...,2.213654
7106,"[Action, Crime, Thriller]",393,The Bride unwaveringly continues on her roarin...,21.533072,/au9lFA5a2ZnBKCzPbZQf00r7J64.jpg,152159461.0,The bride is back for the final cut.,Kill Bill: Vol. 2,7.7,4061.0,The Bride unwaveringly continues on her roarin...,2.221439
1785,"[Romance, Comedy, Crime]",1389,"Meet Jack Foley, a smooth criminal who bends t...",7.349812,/hSL2Ta4ErgAxPEOlkkxZmrTSAET.jpg,77745568.0,Opposites attract.,Out of Sight,6.5,338.0,"Meet Jack Foley, a smooth criminal who bends t...",2.239069
11411,"[Horror, Thriller, Science Fiction]",1562,"In this chilling sequel to 28 Days Later, the ...",12.884191,/kcJ99AtUykDhpzfQOApsViQa3fj.jpg,64238440.0,When days turn to weeks... the horror returns.,28 Weeks Later,6.5,1225.0,"In this chilling sequel to 28 Days Later, the ...",2.267127


#making pivot matrix

## Merging ratings and movies df

In [ ]:
movie_ratings_df = pd.merge(ratings, movies, on = "movieId")
movie_ratings_df

,userId,movieId,rating,timestamp,genres,overview,popularity,poster_path,revenue,tagline,title,vote_average,vote_count,description,baysian_avg
0,1,110,1.0,1425941529,"[Drama, Mystery, Romance]",Red This is the third film from the trilogy by...,7.832755,/77CFEssoKesi4zvtADEpIrSKhA3.jpg,0.0,<NA>,Three Colors: Red,7.8,246.0,<NA>,4.011862
1,1,147,4.5,1425942435,[Drama],"For young Parisian boy Antoine Doinel, life is...",7.268688,/nKVXTcCtOc3Zuu4yCEDMOUyeXfH.jpg,0.0,Angel faces hell-bent for violence.,The 400 Blows,8.0,363.0,"For young Parisian boy Antoine Doinel, life is...",3.588874
2,1,858,5.0,1425941523,"[Comedy, Drama, Romance]",A young boy who tries to set his dad up on a d...,10.234919,/afkYP15OeUOD0tFEmj6VvejuOcz.jpg,227799884.0,"What if someone you never met, someone you nev...",Sleepless in Seattle,6.5,630.0,A young boy who tries to set his dad up on a d...,4.331688
3,1,1246,5.0,1425941556,[Drama],When he loses a highly publicized virtual boxi...,11.697604,/usZMPGxyRsyOpX5nuqHFxH3qlMh.jpg,155721132.0,It ain't over 'til it's over.,Rocky Balboa,6.5,858.0,When he loses a highly publicized virtual boxi...,3.904374
4,1,1968,4.0,1425942148,"[Drama, Comedy, Romance]",Alex Whitman (Matthew Perry) is a designer fro...,6.285574,/e1rzqz7ehOk6uzu7Wkgu4q3L8CK.jpg,29481428.0,What if finding the love of your life meant ch...,Fools Rush In,5.8,133.0,Alex Whitman (Matthew Perry) is a designer fro...,3.821199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10866220,270896,48385,4.0,1257031939,"[Crime, Horror, Science Fiction]",Scientific experiments accidentally revive an ...,3.692449,/4Q4Ft6xUUkvFALH4dgY8am5aCX.jpg,0.0,"The Screen's 300,000 Volt SHOCKER!",Indestructible Man,4.9,7.0,Scientific experiments accidentally revive an ...,3.387730
10866221,270896,48780,5.0,1257031830,[Foreign],A journey into night.,0.479388,/9O1eTVnXvnNUuxj9VML3X8DuLTJ.jpg,0.0,<NA>,Boat,7.6,9.0,<NA>,4.063074
10866222,270896,49530,4.0,1257034436,"[Action, Thriller, Science Fiction]",In the not-too-distant future the aging gene h...,10.324613,/lnYuAr3QOPzvuEFlzpsRUq41IEy.jpg,173930596.0,Time Is Power,In Time,6.7,3512.0,In the not-too-distant future the aging gene h...,3.847906
10866223,270896,54503,4.0,1257033886,"[Action, Foreign]",Lee Yi Min stars as an eager young kung fu stu...,0.342469,/loOl9GCbSzQNq5unvuBowttOyk9.jpg,0.0,<NA>,The Mystery of Chess Boxing,5.9,5.0,<NA>,3.720501


In [ ]:
movie_ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866225 entries, 0 to 10866224
Data columns (total 14 columns):
 #   Column        Dtype  
---  ------        -----  
 0   userId        int64  
 1   movieId       int64  
 2   rating        float64
 3   timestamp     int64  
 4   genres        object 
 5   overview      object 
 6   popularity    float64
 7   poster_path   string 
 8   revenue       float64
 9   tagline       string 
 10  title         string 
 11  vote_average  float64
 12  vote_count    float64
 13  baysian_avg   float64
dtypes: float64(6), int64(3), object(2), string(3)
memory usage: 1.1+ GB


### checking for null values

In [ ]:
for i in movie_ratings_df.columns:
  print(i, movie_ratings_df[i].isnull().any())

userId False
movieId False
rating False
timestamp False
genres False
overview True
popularity False
poster_path False
revenue False
tagline True
title False
vote_average False
vote_count False
baysian_avg False


### checking for duplicate values

In [ ]:
duplicates = movie_ratings_df[movie_ratings_df.duplicated(subset=['userId', 'title'], keep=False)]
print(duplicates)

Empty DataFrame
Columns: [userId, movieId, rating, timestamp, genres, overview, popularity, poster_path, revenue, tagline, title, vote_average, vote_count, baysian_avg]
Index: []


## creating the pivot table

In [ ]:
pivot = movie_ratings_df.pivot_table(index='title', columns='userId', values='rating', )
pivot

# making the scipy matrix
this is used for efficient storage of sparse data

In [ ]:
from scipy.sparse import csr_matrix
matrix = csr_matrix(pivot)
matrix

# movie based collaborative filltering model:
## predict similar movies based on similar user ratings
this content based recommender calculates the cosine similarity between the pivot vector for given movie name and rest of the movies in the pivot table. it then returns the most similar vectors. this means if a movies A and B have both been rated highly by say users x, y, z, then upon prediction for movie A, movie B will be returned and vice versa.
the intuition behind this model is that, we try to find movies rated similarly by users. if movie A was liked by user 1 through 10, and there aslso exists a movie B, liked by 9 of these users, we should recommend movie B to the 10th user. he will most likely like it aas well.

## defining the knn model

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(n_neighbours = 11, algorithm='auto', metric="cosine")
model.fit(matrix)

## making recommendations:

In [ ]:
def movie_collaborative_model(pivot, model, movie_title, num_pred):
  if movie_title not in pivot.index:
    return "Movie not found"

  movie_idx = pivot.index.get_loc(movie_title)
  movie_value_vector = pivot.iloc[movie_idx]
  dist, ind = model.kneighbours(movie_value_vector, n_neighbours = num_pred+1)
  recommendations = [pivot.index[i] for i in ind if i != movie_idx]
  return recommendations

In [ ]:
title = movies['title'].loc(0)
recommendations = movie_collaborative_model(pivot, model, title, 10)
print(recommendations)

# User based collaborative filltering model
## predict similar users based on ratings
in this model we are making the predicitons based on finding similar users. the intuition is once you find 2 users who have similar tastes in movies, any movie user A likes will also be liked by user B

In [ ]:
pivot_t = pivot.T
matrix_t = csr_matrix(pivot_t)

## defining the knn model

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(n_neighbours = 11, algorithm='auto', metric="cosine")
model.fit(matrix_t)

## making recommendations:

In [ ]:
def user_collaborative_model(pivot, df, model, userId, num_pred):
  if userId not in pivot.index:
    return "User not found"

  user_idx = pivot.index.get_loc(userId)
  user_value_vector = pivot.iloc[user_idx]
  dist, ind = model.kneighbours(user_value_vector, n_neighbours = 5)
  recommendations = []
  for i in ind[0]:
    u_idx = pivot.index[i]
    user_movies = df[df["user_id"] == u_idx]
    top_movies = user_movies.sort_values(by="rating", ascending=False).head(num_pred/5)
    recommendations.extend(top_movies["title"].tolist())
  return recommendations

In [ ]:
user = 23
recommendations = user_collaborative_model(pivot_t, movie_ratings_df, model, title, 10)
print(recommendations)

#content based recommendation system
use nlp to get movies with similar summary

## Summary based system
using overview and tagline of the movie to get similar predictions

### Load the spacy model

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 5.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


### Apply preprocessing on description

In [ ]:
movies["description"]

,description
0,<NA>
1,When siblings Judy and Peter discover an encha...
2,A family wedding reignites the ancient feud be...
3,"Cheated on, mistreated and stepped on, the wom..."
4,Just when George Banks has recovered from his ...
...,...
41909,<NA>
41910,Rising and falling between a man and woman. Ri...
41911,<NA>
41912,<NA>


In [ ]:
import spacy
nlp = spacy.load("en_core_web_md")

In [ ]:
def preprocess_text(text):
      if isinstance(text, str):
        doc = nlp(text)
        tokens = [token.lemma_.lower() for token in doc if not token.is_stop and token.is_alpha]
        processed_text = ' '.join(tokens)
        return processed_text
      return np.nan

In [ ]:
movies["description"] = movies["description"].apply(preprocess_text)
movies["description"]

,description
0,NaN
1,sibling judy peter discover enchanted board ga...
2,family wedding reignite ancient feud door neig...
3,cheat mistreat step woman hold breath wait elu...
4,george banks recover daughter wedding receive ...
...,...
41909,NaN
41910,rise fall man woman rise fall man woman
41911,NaN
41912,NaN


### making a tfidf model

In [ ]:
movies["description"] = movies["description"].fillna('')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer()
X = tv.fit_transform(movies["description"]).toarray()

### Using cosine similarity to predict similar movies

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_similar_article(text, vectorizer, X, df, top_n=5):
  if isinstance(text, str):
    preprocessed_text = preprocess_text(text)
    X_text = vectorizer.transform([preprocessed_text])
    similarity_scores = cosine_similarity(X, X_text).flatten()
    top_indices = similarity_scores.argsort()[-top_n:][::-1]
    return df.iloc[top_indices], similarity_scores[top_indices], similarity_scores

In [ ]:
similar_movies, scores, similarity_scores = get_similar_article(movies["description"][1], tv, X, movies,10)
print(scores)
similar_movies

## cast based content recommendation system
using cast, director, genres and keywords to predict similar movies

### Merging movies with credits and keywords dataframe

In [ ]:
movies_credits_keywords = movies.merge(credits, on="movieId")
movies_credits_keywords = movies_credits_keywords.merge(keywords, on="movieId")

### Making a seperate column consisting of cast, directors, genre and keywords.
including director 5 times to give it more weightage

In [ ]:
def get_string_from_list(list):
  return ' '.join(list)

movies_credits_keywords["cast"] = movies_credits_keywords["cast"].apply(get_string_from_list)
movies_credits_keywords["genres"] = movies_credits_keywords["genres"].apply(get_string_from_list)
movies_credits_keywords["keywords"] = movies_credits_keywords["keywords"].apply(get_string_from_list)

movies_credits_keywords["cast_director_genre_keywords"] = movies_credits_keywords["cast"] + movies_credits_keywords["director"] * 5 + movies_credits_keywords["genres"] + movies_credits_keywords["keywords"]


In [ ]:
movies_credits_keywords["cast_director_genre_keywords"][0]

'T o m   H a n k s   T i m   A l l e n   D o n   R i c k l e s   J i m   V a r n e y   W a l l a c e   S h a w nJohn LasseterJohn LasseterJohn LasseterJohn LasseterJohn LasseterAnimation Comedy Familyjealousy toy boy friendship friends rivalry boy next door new toy toy comes to life'

In [ ]:
movies_credits_keywords['cast_director_genre_keywords'].apply(preprocess_text)
movies_credits_keywords["cast_director_genre_keywords"][0]

'T o m   H a n k s   T i m   A l l e n   D o n   R i c k l e s   J i m   V a r n e y   W a l l a c e   S h a w nJohn LasseterJohn LasseterJohn LasseterJohn LasseterJohn LasseterAnimation Comedy Familyjealousy toy boy friendship friends rivalry boy next door new toy toy comes to life'

### Creating a count vectorizer model


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(movies_credits_keywords["cast_director_genre_keywords"]).toarray()

In [ ]:
similar_movies, scores, similarity_scores = get_similar_article(movies_credits_keywords["cast_director_genre_keywords"][0],cv, X, movies_credits_keywords)
print(scores)
similar_movies

## Hybrid content recommendation system

In [ ]:
movies_credits_keywords["overall_detail"] = movies_credits_keywords["description"] + movies_credits_keywords["cast_director_genre_keywords"]*2

### Creating a tfidf vectorizer model


In [ ]:
tv_full = tfidfVectorizer()
X = tv_full.fit_transform(movies_credits_keywords["cast_director_genre_keywords"]).toarray()

In [ ]:
similar_movies, scores, similarity_scores = get_similar_article(movies_credits_keywords["overall_details"][1], tv_full, X, movies_credits_keywords)
print(scores)
similar_movies